In [70]:
import advent
import pandas as pd
import numpy as np

def parse_line(line):
    line = line.split(' ')
    on = 1 if line[0] == 'on' else 0
    coords = line[1].split(',')
    coords = [int(c)  for coord in coords for c in coord[2:].split('..')]
    return [on] + coords

data = pd.DataFrame(advent.get_lines(22, map_fn = parse_line))


In [71]:
cube = np.zeros((101, 101, 101))

def trim(a, b):
    a[0], a[2], a[4] = max(a[0], b[0]), max(a[2], b[0]), max(a[4], b[0])
    a[1], a[3], a[5] = min(a[1]+1, b[1]), min(a[3]+1, b[1]), min(a[5]+1, b[1])
    return a

def assign_to_cube(cube, coords, value, xmin=-50):
    # c is a tuple of xmin,xmax,ymin,ymax,zmin,zmax
    # cube MUST be a cube, e.g. cube.shape[0] == cube.shape[1] == cube.shape[2]
    coords = trim([c-xmin for c in coords], (0, cube.shape[0]))
    cube[coords[0]:coords[1], coords[2]:coords[3], coords[4]:coords[5]] = value
    return cube

for coords in list(data.itertuples()):
    cube = assign_to_cube(cube, coords[2:], coords[1])

cube.sum()

642125.0

In [ ]:
# Part 2

# The obvious idea here is to not keep track of every cuboid in a giant 100kx100kx100k
# 3d matrix, but just to keep track of the corners only.
# Problem being, cuboids overlapping creates all kind of weird not-square like objects with many corners
# There are several complicated ways I can think of dealing with this, but the idea I ended up trying was:
# Keep track of a list of turned-on cuboids
# When two cuboids overlap, split one of them up into 8 cuboids, where 1 of the 8 covers the overlap
# And replace the 1 with the 8. Then overlap becomes much easier to compute: just remove the 1 overlap cuboid,
# and treat the 2nd cuboid as if the overlap doesn't exist (after all we just threw it away)

# The goal is that in our list of turned-on cuboids, after every step, none of them should overlap anywhere

from typing import NamedTuple, Optional
import itertools
import math

def clamp(x: tuple[int, int], c: float) -> list[tuple[int, int]]:
    # return (xmin, d), (d+1, xmax) such that d<c<d+1
    # However, if c < xmin or c > xmax, just return (xmin, xmax)
    xmin, xmax = x
    assert xmin <= xmax
    if c < xmin or c > xmax: return [(xmin, xmax)]
    return [(xmin, math.floor(c)), (math.ceil(c), xmax)]

def pick_point_in_interior(this, other):
    # Picks a point out of either otherxmin, otherxmax
    # By preferring one that is thisxmin < _ < thisxmax
    # if neither matches that, prefer one that is thisxmin <= _ <= thisxmax
    # If neither matches that, just return otherxmax
    thisxmin, thisxmax = this
    otherxmin, otherxmax = other
    if thisxmin < otherxmin < thisxmax: return otherxmin
    elif thisxmin < otherxmax < thisxmax: return otherxmax
    elif thisxmin <= otherxmin <= thisxmax: return otherxmin
    return otherxmax


# We are using a class just because there is going to be so much index messing around. any structure will help
coordinate = tuple[int|float, int|float, int|float]

class cuboid(NamedTuple):
    # Corners of the cuboid, inclusive
    x: tuple[int, int]
    y: tuple[int, int]
    z: tuple[int, int]

    def has_area(self) -> bool:
        return self.x[0] <= self.x[1] and self.y[0] <= self.y[1] and self.z[0] <= self.z[1]

    def size(self) -> int:
        return (self.x[1] - self.x[0] + 1) * (self.y[1] - self.y[0] + 1) * (self.z[1] - self.z[0] + 1)

    def contains(self, coord: coordinate) -> bool:
        return (self.x[0] <= coord[0] <= self.x[1]) and \
               (self.y[0] <= coord[1] <= self.y[1]) and \
               (self.z[0] <= coord[2] <= self.z[1])
    
    def contains_cuboid(self, other: 'cuboid') -> bool:
        # Returns true if other is completely contained in self
        return self.x[0] <= other.x[0] and other.x[1] <= self.x[1] and \
                self.y[0] <= other.y[0] and other.y[1] <= self.y[1] and \
                self.z[0] <= other.z[0] and other.z[1] <= self.z[1]

    def nudge_corner_towards_outside(self, coord: coordinate) -> coordinate:
        # yeah kinda weird but I need this to call split later
        # Does what it says: takes a corner and nudges by adding or subtracting 0.5
        # So that the resulting coordinate is not in the cuboid
        return (
            coord[0] - 0.5 if coord[0] == self.x[0] else coord[0] + 0.5,
            coord[1] - 0.5 if coord[1] == self.y[0] else coord[1] + 0.5,
            coord[2] - 0.5 if coord[2] == self.z[0] else coord[2] + 0.5
        )
    
    #def corners(self) -> list[coordinate]:
    #    return list(itertools.product(self.x, self.y, self.z))

    def split(self, middle: coordinate) -> list['cuboid']:
        # middle here should be something like (4.5, 3.5, 2.5), aka NOT INTEGERS
        # Returns 8 cuboids: the 8 cuboids on each x/y/z direction of the middle
        # if middle is not inside self, this may return less than 8 cuboids...
        # (as if this problem wasn't hard enough :) )
        xclamp = clamp(self.x, middle[0])
        yclamp = clamp(self.y, middle[1])
        zclamp = clamp(self.z, middle[2])
        result = []
        for xyz in itertools.product(xclamp, yclamp, zclamp):
            result.append(cuboid(xyz[0], xyz[1], xyz[2]))
        return result
    
    def has_overlap(self, other: 'cuboid') -> bool:
        xoverlap = other.x[1] >= self.x[0] and self.x[1] >= other.x[0]
        yoverlap = other.y[1] >= self.y[0] and self.y[1] >= other.y[0]
        zoverlap = other.z[1] >= self.z[0] and self.z[1] >= other.z[0]
        return xoverlap and yoverlap and zoverlap
    
    def split_point(self, other: 'cuboid') -> coordinate:
        # The idea of this function: we want to split this cube somewhere
        # Such that 7 of the 8 resulting cuboids do not overlap with other.
        # That way, we can keep reducing the overlapping volume with other

        # if other has a corner that is inside self, return it
        # If there are multiple, just return any (hopefully doesn't matter?)
        # If there is no corner of other that is inside self, return None
        
        # However! We want the corner of other that is most 'inside' self.
        # aka: for x-dim, if other.x[0] is xmin, take other.x[1]. etcetera
        # if self.x and other.x are equal, any corner will work
        proposed_x = pick_point_in_interior(self.x, other.x)
        proposed_y = pick_point_in_interior(self.y, other.y)
        proposed_z = pick_point_in_interior(self.z, other.z)
        proposed = (proposed_x, proposed_y, proposed_z)
        return other.nudge_corner_towards_outside(proposed)

cube = cuboid(x=(0,1), y=(2,3), z=(4,5))
#corners = cube.corners()
n = cube.nudge_corner_towards_outside((1, 2, 5))
print(n)
print(cube.split((0.5, 2.5, 5.5)))
print(cube.split_point(cuboid((0,2),(2,4),(4,6))))

(1.5, 1.5, 5.5)
[cuboid(x=(0, 0), y=(2, 2), z=(4, 5)), cuboid(x=(0, 0), y=(3, 3), z=(4, 5)), cuboid(x=(1, 1), y=(2, 2), z=(4, 5)), cuboid(x=(1, 1), y=(3, 3), z=(4, 5))]
(-0.5, 1.5, 3.5)


In [ ]:
on_cubes: list[cuboid] = []
import tqdm

for coords in tqdm.tqdm(list(data.itertuples())):
    turn_on = coords[1] == 1 # this only determines whether we add the cube to the list at the end
    # Otherwise the goal is the same: remove all overlap between on_cubes and cube
    cube = cuboid((coords[2], coords[3]), (coords[4], coords[5]), (coords[6], coords[7]))
    assert cube.has_area()
    #print(cube)
    overlap_found = True
    while overlap_found:
        overlap_found = False
        for to_split in on_cubes:
            if not to_split.has_overlap(cube): continue # there is no overlap
            overlap_found = True
            middle = to_split.split_point(cube)
            #print(f"Overlap found! {cube} with {c}, {middle}")
            #print(to_split, cube, middle)
            new_cubes = to_split.split(middle)
            on_cubes.remove(to_split)
            for new_cube in new_cubes:
                if not cube.contains_cuboid(new_cube) and new_cube.has_area():
                    on_cubes.append(new_cube)
            break # try again :D
    if turn_on: on_cubes.append(cube)
    #print(on_cubes)
    #print('-----------------')
    #print(sum([cube.size() for cube in on_cubes]))
    #print('-----------------')

print(sum([cube.size() for cube in on_cubes]))
# 1993810700040032 too high
# 

100%|██████████| 420/420 [00:08<00:00, 48.17it/s]  

1235164413198198
